In [0]:
%pip install openpyxl
import pandas as pd
from pathlib import Path
import openpyxl
from pyspark.sql import DataFrame as SparkDataFrame
import re

#Import data and create tables

In [0]:
main_directory = Path('/Volumes/workspace/default/dopravní_nehody_čr')
all_dataframes = {}
for folder in main_directory.rglob('*'):
   if folder.is_dir():
       for file in folder.rglob('*.xlsx'):
           jmeno_df = f"{file.stem}_{folder.stem}"
           all_dataframes[jmeno_df] = pd.read_excel(file)



In [0]:
all_dataframes['Inasledky_2023']['p59d'] = all_dataframes['Inasledky_2023']['p59d'].astype(str)

In [0]:
for key, value in all_dataframes.items():
    if not isinstance(value, SparkDataFrame):
        all_dataframes[key] = spark.createDataFrame(value)
    print(f"{key}: done")

In [0]:
for key, value in all_dataframes.items():
    value.write.format('delta').mode('overwrite').saveAsTable(key)

In [0]:
%sql
CREATE OR REPLACE table nehody AS 
SELECT * FROM default.Inehody_2023
UNION ALL
SELECT * FROM default.Inehody_2024
UNION ALL
SELECT * FROM default.Inehody_2025; 

CREATE OR REPLACE table vozidla AS 
SELECT * FROM default.IVozidla_2023
UNION ALL
SELECT * FROM default.IVozidla_2024
UNION ALL
SELECT * FROM default.IVozidla_2025; 

CREATE OR REPLACE table chodci AS 
SELECT * FROM default.Ichodci_2023 
UNION ALL
SELECT * FROM default.Ichodci_2024 
UNION ALL
SELECT * FROM default.Ichodci_2025;


CREATE OR REPLACE table nasledky AS 
SELECT * FROM default.Inasledky_2023
UNION ALL
SELECT * FROM default.Inasledky_2024
UNION ALL
SELECT * FROM default.Inasledky_2025;

CREATE OR REPLACE table gps AS 
SELECT * FROM default.IntGPS_2023
UNION ALL
SELECT * FROM default.IntGPS_2024
UNION ALL
SELECT * FROM default.IntGPS_2025;



In [0]:
%sql
DROP TABLE IF EXISTS IVozidla_2023;
DROP TABLE IF EXISTS Ichodci_2023;
DROP TABLE IF EXISTS Inasledky_2023;
DROP TABLE IF EXISTS Inehody_2023;
DROP TABLE IF EXISTS IntGPS_2023;
DROP TABLE IF EXISTS IVozidla_2024;
DROP TABLE IF EXISTS Ichodci_2024;
DROP TABLE IF EXISTS Inasledky_2024;
DROP TABLE IF EXISTS Inehody_2024;
DROP TABLE IF EXISTS IntGPS_2024;
DROP TABLE IF EXISTS IVozidla_2025;
DROP TABLE IF EXISTS Ichodci_2025;
DROP TABLE IF EXISTS Inasledky_2025;
DROP TABLE IF EXISTS Inehody_2025;
DROP TABLE IF EXISTS IntGPS_2025;

In [0]:
%sql
DELETE FROM nehody
WHERE p1 == 161425000002 AND p8 == 6;

In [0]:
%sql
SELECT * FROM nehody
WHERE p1 is NULL

In [0]:
%sql
ALTER TABLE nehody ALTER COLUMN p1 SET NOT NULL;
ALTER TABLE nehody ADD PRIMARY KEY(p1);
ALTER TABLE vozidla ADD FOREIGN KEY(p1) REFERENCES nehody(p1);
ALTER TABLE gps ADD FOREIGN KEY(p1) REFERENCES nehody(p1);
ALTER TABLE chodci ADD FOREIGN KEY(p1) REFERENCES nehody(p1);
ALTER TABLE nasledky ADD FOREIGN KEY(p1) REFERENCES nehody(p1);

In [0]:
%sql
COMMENT ON COLUMN chodci.p1 IS 'identifikační číslo nehody';
COMMENT ON COLUMN chodci.p29 IS 'kategorie chodce';
COMMENT ON COLUMN chodci.p29a IS 'reflexní prvky u chodce';
COMMENT ON COLUMN chodci.p29b IS 'chodec na osobním přepravníku';
COMMENT ON COLUMN chodci.p30 IS 'stav chodce';
COMMENT ON COLUMN chodci.p30a IS 'alkohol u chodce přítomen';
COMMENT ON COLUMN chodci.p30b IS 'druh drogy u chodce';
COMMENT ON COLUMN chodci.p31 IS 'chování chodce';
COMMENT ON COLUMN chodci.p32 IS 'situace v místě nehody';
COMMENT ON COLUMN chodci.p33c IS 'pohlaví zraněné osoby';
COMMENT ON COLUMN chodci.p33d IS 'věk zraněného chodce';
COMMENT ON COLUMN chodci.p33e IS 'státní příslušnost (stát) zraněného';
COMMENT ON COLUMN chodci.p33f IS 'poskytnutí první pomoci zraněnému';
COMMENT ON COLUMN chodci.p33g IS 'následky na zraněném';

COMMENT ON COLUMN nasledky.p1 IS 'identifikační číslo nehody';
COMMENT ON COLUMN nasledky.id_vozidla IS 'identifikační číslo vozidla u nehody';
COMMENT ON COLUMN nasledky.p59a IS 'označení osoby';
COMMENT ON COLUMN nasledky.p59b IS 'bližší označení osoby';
COMMENT ON COLUMN nasledky.p59c IS 'pohlaví osoby';
COMMENT ON COLUMN nasledky.p59d IS 'věk osoby';
COMMENT ON COLUMN nasledky.p59e IS 'státní příslušnost (stát)';
COMMENT ON COLUMN nasledky.p59f IS 'poskytnutí první pomoci';
COMMENT ON COLUMN nasledky.p59g IS 'následky';

COMMENT ON COLUMN nehody.p1 IS 'identifikační číslo nehody';
COMMENT ON COLUMN nehody.p2a IS 'den, měsíc, rok';
COMMENT ON COLUMN nehody.p2b IS 'čas ';
COMMENT ON COLUMN nehody.p4a IS 'kraj';
COMMENT ON COLUMN nehody.p4b IS 'okres';
COMMENT ON COLUMN nehody.p4c IS 'útvar';
COMMENT ON COLUMN nehody.p5a IS 'lokalita nehody';
COMMENT ON COLUMN nehody.p6 IS 'druh nehody';
COMMENT ON COLUMN nehody.p7 IS 'druh srážky jedoucích vozidel';
COMMENT ON COLUMN nehody.p8 IS 'druh pevné překážky';
COMMENT ON COLUMN nehody.p8a IS 'druh zvěře / zvířete';
COMMENT ON COLUMN nehody.p9 IS 'charakter nehody';
COMMENT ON COLUMN nehody.p10 IS 'zavinění nehody';
COMMENT ON COLUMN nehody.p11 IS 'alkohol u viníka nehody přítomen';
COMMENT ON COLUMN nehody.p11a IS 'drogy u viníka nehody';
COMMENT ON COLUMN nehody.p12 IS 'hlavní příčiny nehody';
COMMENT ON COLUMN nehody.p13a IS 'usmrceno osob';
COMMENT ON COLUMN nehody.p13b IS 'těžce zraněno osob';
COMMENT ON COLUMN nehody.p13c IS 'lehce zraněno osob';
COMMENT ON COLUMN nehody.p14 IS 'celková hmotná škoda v kč';
COMMENT ON COLUMN nehody.p15 IS 'druh povrchu vozovky';
COMMENT ON COLUMN nehody.p16 IS 'stav povrchu vozovky v době nehody';
COMMENT ON COLUMN nehody.p17 IS 'stav komunikace';
COMMENT ON COLUMN nehody.p18 IS 'povětrnostní podmínky v době nehody';
COMMENT ON COLUMN nehody.p19 IS 'viditelnost';
COMMENT ON COLUMN nehody.p20 IS 'rozhledové poměry';
COMMENT ON COLUMN nehody.p21 IS 'dělení komunikace';
COMMENT ON COLUMN nehody.p22 IS 'situování nehody na komunikaci';
COMMENT ON COLUMN nehody.p23 IS 'řízení provozu v době nehody';
COMMENT ON COLUMN nehody.p24 IS 'místní úprava přednosti v jízdě';
COMMENT ON COLUMN nehody.p27 IS 'specifická místa a objekty v místě nehody';
COMMENT ON COLUMN nehody.p28 IS 'směrové poměry';
COMMENT ON COLUMN nehody.p34 IS 'počet zúčastněných vozidel';
COMMENT ON COLUMN nehody.p35 IS 'místo dopravní nehody';
COMMENT ON COLUMN nehody.p36 IS 'druh pozemní komunikace';
COMMENT ON COLUMN nehody.p37 IS 'číslo pozemní komunikace';
COMMENT ON COLUMN nehody.p38 IS 'CHYBÍ DOKUMENTACE';
COMMENT ON COLUMN nehody.p39 IS 'druh křižující komunikace';

COMMENT ON COLUMN gps.p1 IS 'identifikační číslo nehody';
COMMENT ON COLUMN gps.d IS 'souřadnice x';
COMMENT ON COLUMN gps.e IS 'souřadnice y';
COMMENT ON COLUMN gps.h IS 'město';
COMMENT ON COLUMN gps.i IS 'ulice';
COMMENT ON COLUMN gps.j IS 'číslo popisné';
COMMENT ON COLUMN gps.k IS 'typ komunikace';

COMMENT ON COLUMN vozidla.p1 IS 'identifikační číslo nehody';
COMMENT ON COLUMN vozidla.id_vozidla IS 'identifikační číslo vozidla u nehody';
COMMENT ON COLUMN vozidla.p44 IS 'druh vozidla';
COMMENT ON COLUMN vozidla.p45a IS 'výrobní značka motorového vozidla';
COMMENT ON COLUMN vozidla.p45b IS 'údaje o vozidle';
COMMENT ON COLUMN vozidla.p45d IS 'druh pohonu / paliva';
COMMENT ON COLUMN vozidla.p45f IS 'druh pneumatik na vozidle';
COMMENT ON COLUMN vozidla.p47 IS 'rok výroby vozidla';
COMMENT ON COLUMN vozidla.p48a IS 'charakteristika vozidla';
COMMENT ON COLUMN vozidla.p48b IS 'CHYBÍ DOKUMENTACE';
COMMENT ON COLUMN vozidla.p49 IS 'smyk';
COMMENT ON COLUMN vozidla.p50a IS 'vozidlo po nehodě';
COMMENT ON COLUMN vozidla.p50b IS 'únik provozních, přepravovaných hmot';
COMMENT ON COLUMN vozidla.p51 IS 'způsob vyproštění osob z vozidla';
COMMENT ON COLUMN vozidla.p52 IS 'směr jízdy nebo postavení vozidla';
COMMENT ON COLUMN vozidla.p53 IS 'škoda na vozidle';
COMMENT ON COLUMN vozidla.p55a IS 'kategorie řidiče';
COMMENT ON COLUMN vozidla.p55b IS 'CHYBÍ DOKUMENTACE';
COMMENT ON COLUMN vozidla.p56 IS 'CHYBÍ DOKUMENTACE';
COMMENT ON COLUMN vozidla.p57 IS 'stav řidiče';
COMMENT ON COLUMN vozidla.p58 IS 'vnější ovlivnění řidiče';

#Adjust documentation data to usable form

In [0]:
%sql
ALTER TABLE documentation
DROP COLUMN _c4, _c5, _c6, _c7, _c8, _c9;

In [0]:
%sql
SELECT * FROM documentation



In [0]:
documentation_df = spark.table("workspace.default.documentation")
df_dokumentace = documentation_df.toPandas()

In [0]:
df_dokumentace.head(15)

In [0]:

for row_number, row in enumerate(df_dokumentace['Položka']):
    if re.search("^p", str(row)):
        df_dokumentace.loc[row_number,'Formulář DN'] = df_dokumentace['Položka'][row_number]
df_dokumentace['Formulář DN'] = df_dokumentace['Formulář DN'].ffill()
df_dokumentace = df_dokumentace.rename(columns={
    'Formulář DN': 'formular_DN',
    'Bližší definice': 'blizsi_definice',
    'Položka': 'polozka',
    'Popis': 'popis'
})

df_dokumentace.head(15)

In [0]:
documentation_spark = spark.createDataFrame(df_dokumentace)
documentation_spark.write.format('delta').mode('overwrite').saveAsTable('dokumentace')

In [0]:
%sql

CREATE OR REPLACE TABLE dokumentace_cz AS
SELECT * FROM dokumentace

In [0]:
%sql
SELECT * FROM dokumentace_cz

In [0]:
%sql
DESCRIBE nehody;

SLOUPCE S POČTY
  
p13a - USMRCENO OSOB  
p13b - TĚŽCE ZRANĚNO OSOB  
p13c - LEHCE ZRANĚNO OSOB  
p14 - CELKOVÁ HMOTNÁ ŠKODA v Kč * 100  
p33d - VĚK CHODCE  
p34 - POČET ZÚČASTNĚNÝCH VOZIDEL  
p53 - ŠKODA NA VOZIDLE * 100   
p59d - VĚK OSOBY  
